<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

## Demonstration of usage of mapping between OpenScale metrics and Integrated System (like OpenPages) metrics when sending metrics to the Integrated System

This notebook should be run in a Watson Studio project, using **IBM Runtime 22.1 on Python 3.9 XS** runtime environment. **If you are viewing this in Watson Studio and do not see the required runtime env in the upper right corner of your screen, please update the runtime now.** It demonstrates an update to the payload of the `'integrated_system_metrics'` endpoint currently being used for the 'Send to OpenPages' functionality. A new attribute `'integrated_metrics'` has been added to the payload which includes mapping between OpenScale metrics and metrics already created in an integrated system like OpenPages. The rationale behind it to provide convenience to the user to re-use metrics already created in the integrated system.

`'integrated_metrics'` attribute example:
```
integrated_metrics": [{
		"integrated_system_type": "open_pages",
		"mapped_metrics": [{
				"internal_metric_id": "Age",
				"external_metric_id": "7789"
		}]
        
}]
```

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

## Credentials for IBM Cloud services

### Retrieve your IBM Cloud API key

1.	From the IBM Cloud toolbar, click your Account name, such as <Your user name>’s Account.
1.	From the Manage menu, click Access (IAM).
1.	In the navigation bar, click IBM Cloud API keys.
1.	Click the Create an IBM Cloud API key button.
1.	Type a name and description and then click Save.
1.	Copy the newly created API key and paste it into your notebook in the following **CLOUD_API_KEY** code box, which is the first code box.

    Note: replace everything between the two sets of double quotation marks (").

In [ ]:
CLOUD_API_KEY = "XXXX"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

### Create the OpenScale SDK Client instance

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient

service_credentials = {
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

authenticator = IAMAuthenticator(apikey=service_credentials['apikey'])

wos_client = APIClient(authenticator=authenticator)
wos_client.version

### Listing the datamarts

In [ ]:
wos_client.data_marts.show()

### Find the data mart to use

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["hostname"],
                        username=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["username"],
                        password=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["password"],
                        db=DB_CREDENTIALS["connection"]["postgres"]["database"],
                        port=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["port"],
                        ssl=True,
                        sslmode=DB_CREDENTIALS["connection"]["postgres"]["query_options"]["sslmode"],
                        certificate_base64=DB_CREDENTIALS["connection"]["postgres"]["certificate"]["certificate_base64"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))

### List the subscriptions and find the specific subscription to use for this notebook

In [ ]:
wos_client.subscriptions.show()

### Select the subscription whose metrics are to be published to OpenPages and replace <subscription_id> below

In [ ]:
subscription_id = "<subscription_id>"

### Let's print the subscription details

In [ ]:
print(wos_client.subscriptions.get(subscription_id = subscription_id).result)

### List all monitor instances

In [ ]:
monitor_instances = wos_client.monitor_instances.list(target_target_id=subscription_id).result.monitor_instances
monitor_instance_id = None
for monitor_instance in monitor_instances:
    monitor_def_id = monitor_instance.entity.monitor_definition_id
    print(monitor_def_id + ' instance id: ' + monitor_instance.metadata.id)
    if monitor_def_id == 'mrm':
        monitor_instance_id = monitor_instance.metadata.id    

### MRM metrics for the selected subscription below

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=monitor_instance_id)

### Instructions on how to obtain OpenPages metric IDs associated with an OpenPages model

There are two ways to obtain the OpenPages metric IDs for a particular OpenPages model:

1. UI Route:


- To get the ID of a particular metric, go to OpenPages model UI and choose a metric under the 'OpenScale Metrics' section. 
- The Metric ID is the number at the end of the URL. For example, for the URL of the form '.../grc/task-view/7779', `7779` is the metric ID.
- Check the Metric description. The OpenScale metric name is the one under single quotes. For example, for the description `Watson OpenScale drift metric for 'data_drift_magnitude'`, the OpenScale metric here is `data_drift_magnitude`.


2. The API route is described in the next cell.

In [ ]:
# The method below fetches all the Metric IDs associated with the provided OpenPages model using OpenPages APIs. 
# The method definition is: get_all_metrics_for_model(model_name, openpages_url, username, password)

# Method definition below --

import requests
import base64
import json

def get_all_metrics_for_model(model_name, openpages_url, username, password):
    
    openpages_url = openpages_url.rstrip("/") + "/grc/api/query"
    
    # Prepare authorization token
    token = base64.b64encode(bytes('{0}:{1}'.format(username, password), 'utf-8')).decode("ascii")
    
    header = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": "Basic {0}".format(token)
    }
    
    # Prepare post payload
    get_id_payload = {
        "statement": "SELECT [Model].[Resource ID] FROM [Model] WHERE [Model].[Name]='{0}'".format(model_name),
        "skipCount": 0
    }
    
    print("Sending request to fetch model ID given model name")
    
    response = requests.post(openpages_url, json=get_id_payload, headers=header).json()
    
    model_id = None
    if response is not None:
        if response.get("rows") is not None:
            rows = response.get("rows")
            if len(rows) != 0:
                fields = rows[0].get("fields")
                if fields is not None:
                    field = fields.get("field")
                    if len(field) != 0:
                        model_id = field[0]["value"]
                        
    if model_id is None:
        print("Model ID not found.")
        print("--------------------------------------------------------------------------")
        return {}
    
    print("Model ID fetched: " + model_id)
    print("--------------------------------------------------------------------------")
    
    # Fetch the metric IDs with the description for the OpenPages model
    get_metrics_payload = {
        "statement": "SELECT [Metric].[Resource ID], [Metric].[Description] FROM [Model] JOIN [Metric] ON PARENT([Model]) WHERE [Model].[Resource ID]='{0}' AND [Metric].[MRG-Metric-Shared:Metric Type]='Watson OpenScale'".format(model_id),
        "skipCount": 0
    }
    
    metric_description_id_mapping = {}
    
    print("Sending request to fetch all metrics associated with the model.")
    
    response = requests.post(openpages_url, json=get_metrics_payload, headers=header).json()
    metric_id_list = []
    metric_description_list = []
    
    if response is not None:
        if response.get("rows") is not None:
            rows = response.get("rows")
            if len(rows) != 0:
                for i in range(len(rows)):
                    fields = rows[i].get("fields")
                    if fields is not None:
                        field = fields.get("field")
                        for val in field:
                            value = val.get("value")
                            if val.get("name") == "Resource ID":
                                metric_id_list.append(value)
                            else:
                                metric_description_list.append(value)
        
    metric_description_id_mapping = {description:id for description, id in zip(metric_description_list, metric_id_list)}
    print("Fetched all metrics")
    print("--------------------------------------------------------------------------")
    
    return metric_description_id_mapping

In [ ]:
# The method definition is: get_all_metrics_for_model(model_name, openpages_url, username, password)
# Please replace 'openpages_model_name', 'openpages_url', 'username', 'password' below appropriately.
# Example <openpages_model_name>: 'MOD_0000096'
# Example <openpages_url>: 'https://softlayer-dev1.op-ibm.com'
# If the response is an empty dictionary, please re-check the attributes specified.
# ----------------------------------------------------------------------------------------------------

openpages_model_name = "MOD_XXXXXX"
openpages_url = "XXXX"
username = "XXX"
password = "XXX"

openpages_metric_to_id_mapping = get_all_metrics_for_model(openpages_model_name, openpages_url, username, password)

print(json.dumps(openpages_metric_to_id_mapping, indent=1))

### Method to fetch all available OpenPages metric values for the metric above

In [ ]:
# The method below fetches all the Metric values associated with a particular metric ---

def get_metric_values_for_a_metric(openpages_metric_id, openpages_url, username, password):
    openpages_url = openpages_url.rstrip("/") + "/grc/api/query"
    
    # Prepare authorization token
    token = base64.b64encode(bytes('{0}:{1}'.format(username, password), 'utf-8')).decode("ascii")
    
    header = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": "Basic {0}".format(token)
    }
    
    # Prepare post payload
    get_id_payload = {
        "statement": "SELECT [MetricValue].[Name], [MetricValue].[Description] FROM [Metric] JOIN [MetricValue] ON PARENT([Metric]) WHERE [Metric].[Resource ID]='{0}'".format(openpages_metric_id),
        "skipCount": 0
    }
    
    print("Sending request to fetch metric values")
    
    response = requests.post(openpages_url, json=get_id_payload, headers=header).json()
    
    metric_value_name_list = []
    metric_value_description_list = []
    if response is not None:
        if response.get("rows") is not None:
            rows = response.get("rows")
            if len(rows) != 0:
                for i in range(len(rows)):
                    fields = rows[i].get("fields")
                    if fields is not None:
                        field = fields.get("field")
                        for val in field:
                            value = val.get("value")
                            if val.get("name") == "Name":
                                metric_value_name_list.append(value)
                            else:
                                metric_value_description_list.append(value)
                                
    print("Fetched all metric values for the provided metric")
    print("--------------------------------------------------------------------------")
                        
    return zip(metric_value_name_list, metric_value_description_list)

In [ ]:
# Check if there are any existing metric value for a metric before mapping:

openpages_metric_id = '<openpages_metric_id>'
metric_values = get_metric_values_for_a_metric(openpages_metric_id, openpages_url, username, password)
print(str(list(metric_values)))

### Sample JSON which shows the selection of metrics to be sent and their mapping to OpenPages metrics.

The json is of the form:
```
{
  "metrics": [
    {
      "type": "quality",
      "measures": [
        "recall"
      ],
	"integrated_metrics": [{
		"integrated_system_type": "open_pages",
		"mapped_metrics": [{
				"internal_metric_id": "recall",
				"external_metric_id": "13758"
			}]
	}],
  "send_report": false
}]}
```

- `'integrated_metrics'` is an optional attribute. 
- The `'internal_metric_id'` is the OpenScale metric name (for example 'Age') and the `'external_metric_id'` is the metric ID of the metric already existing in the integrated system which the user wants to map to an OpenScale metric (for example '7789').
- Only those metrics are sent to OpenPages for which the user has decided the mapping.

### Call the MRM Send to OpenPages API

Note: Please initiate an MRM evaluation if not done already 

### Get the IAM token

In [ ]:
# Generate the bearer token 
get_bearer_token_header = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

params = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": CLOUD_API_KEY
}

print("Generating bearer token.....")
bearer_token = requests.post("https://iam.ng.bluemix.net/oidc/token", 
                             headers=get_bearer_token_header, params=params).json()["access_token"]
print("Generated bearer token successfully.")

In [ ]:
## Fetch a measurement and fetch monitor_instance_id and run_id for the monitor run from the measurement
header = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": "Bearer {0}".format(bearer_token)
}

print("Fetching MRM measurement....")

get_measurements_endpoint = 'https://api.aiopenscale.cloud.ibm.com/openscale/' + \
                data_mart_id + '/v2/measurements?target_id=' + subscription_id + \
                '&target_type=subscription&monitor_definition_id=mrm&recent_count=1&format=full'

get_measurements_response = requests.get(get_measurements_endpoint, headers=header).json()
measurements = get_measurements_response["measurements"]

if len(measurements) == 0:
    print("MRM evaluation has not completed.")
    print("--------------------------------------")
else:
    print("Fetched MRM measurement successfully.")
    print("--------------------------------------")
    measurement = measurements[0]
    monitor_instance_id = measurement["entity"]["monitor_instance_id"]
    monitor_run_id = measurement["entity"]["run_id"]
    
    print("Monitor instance ID fetched: " + monitor_instance_id)
    print("Monitor run ID fetched: " + monitor_run_id)
    
    post_payload = {
      "metrics": [{
          "type": "quality",
          "measures": [
            "recall"
          ],
        "integrated_metrics": [{
            "integrated_system_type": "open_pages",
            "mapped_metrics": [{
                    "internal_metric_id": "recall",
                    "external_metric_id": openpages_metric_id
                }]
        }],
      "send_report": False
    }]}

    # Send metrics to openpages
    print("Sending metrics to OpenPages")    
    
    send_to_op_endpoint = "https://api.aiopenscale.cloud.ibm.com/openscale/" + data_mart_id + \
            "/v2/monitoring_services/mrm/monitor_instances/" + monitor_instance_id + \
            "/runs/" + monitor_run_id + "/integrated_system_metrics"
    
    send_to_op_response = requests.put(send_to_op_endpoint, json=post_payload, headers=header).json()
    
    print("Sent metrics to OpenPages successfully.")
    print("--------------------------------------")

In [ ]:
# Wait for couple of seconds to make sure the metrics are indeed send to OpenPages and query and list them.

metric_values = get_metric_values_for_a_metric(openpages_metric_id, openpages_url, username, password)
print(str(list(metric_values)))

As it can be seen, the OpenPages metric value has been created for the mapped OpenPages metric